# 自言自语
IG啊IG，今年你还有机会吗\
效果图：
![](https://ai-studio-static-online.cdn.bcebos.com/719712e91eae44269498c55ef3786a93de1882155c284b72b6a9f57c056262c1)

# 安装paddlehub环境
安装paddlehub

In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!pip install paddlehub==2.0.0b2

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 204kB 13.0MB/s eta 0:00:01
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


# 数据准备
准备了几张照片（19春季赛的定妆照
![](https://ai-studio-static-online.cdn.bcebos.com/ed6bf73a8cdd431fac8793f5c3ca08004f261c550ef34871901423bfb38b374c)
![](https://ai-studio-static-online.cdn.bcebos.com/760a10c796f949a6a0b03130ab0de17cc01a74c7313344c48706c791bb93b77c)
![](https://ai-studio-static-online.cdn.bcebos.com/afe01dbf777e42dcba7a95d5805c5069752fa03b5fb8462d824b617df6ff4318)
![](https://ai-studio-static-online.cdn.bcebos.com/eaf61b2270b84b31aaf7a099050d19827bda1001d6dc431d8f422c995a23d3cb)
![](https://ai-studio-static-online.cdn.bcebos.com/26d1b9295d664050a2a7308699c82949560797042e3a432b8c9f6d113bc1564b)
![](https://ai-studio-static-online.cdn.bcebos.com/ceab6b7cd2b84fa98cd308450f223a7df7d6d61d6ca34044ac832fdb24a09e92)

# 导入U2Net_Portrait模型
项目采用了U2Net_Portrait网络。从PaddleHub里导入了预训练模型

In [ ]:
import paddlehub as hub
import os
import cv2

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# 模型加载
# use_gpu：是否使用GPU进行预测
for f in os.listdir('./18ji'):
    model = hub.Module(name='U2Net_Portrait')
    # img = cv2.imread(os.path.join('./18ji', f))
    # result = model.Cartoon_GEN(images=[img], output_dir='./output_frames')
    result = model.Portrait_GEN(images=[img], output_dir='./output_frames2')
    # print(result[0].shape)
    cv2.imwrite(f'./18ji_output/{f}', result[0])

146.0


![](https://ai-studio-static-online.cdn.bcebos.com/2625bf36fb154b588f17b88c40c3dff29a8c67992c784670afdd87ebf504ff56)
![](https://ai-studio-static-online.cdn.bcebos.com/ed638b05becc47e4b2740cdb148156fbe1aae83b11ff4100bdda333e0f9c15bf)
![](https://ai-studio-static-online.cdn.bcebos.com/556c195e315549dbbd519a00a88b3fa012cf9de569f04d87984e432d5d5ba6d4)
![](https://ai-studio-static-online.cdn.bcebos.com/22ea6474e5c14c8dbed57467e51b49c06f78b54e951d494aa2614d56d0aa433d)
![](https://ai-studio-static-online.cdn.bcebos.com/8a2e7fee27c9477f942f1cf6486511bd554634e48a054253b5837c226f9c9d34)
![](https://ai-studio-static-online.cdn.bcebos.com/92979f19e44e4fddb498b9cd8767b58150fca99a97544f5bae22f714eb82438d)
![](https://ai-studio-static-online.cdn.bcebos.com/aade7d1b9f19485e87c70c583f7199ea836eabcd81b54255a1d3ebb3986ea4ff)

\（呜呜呜shy哥好帅我要放两张）


# 合成为一张图

In [16]:
import cv2
import numpy as np
import os

root = './18ji_output'
w, h = 128, 128
cnt_h = 4
cnt_w = 5
res = np.zeros((h * cnt_h, w * cnt_w, 3))
now_h = 0
now_w = 0
for path in os.listdir(root):
    path = os.path.join(root, path)
    img = cv2.resize(cv2.imread(path), (w, h))
    # cv2.imwrite('f.png', img)
    # print(now_h * h, (now_h + 1) * h)
    res[now_h * h: (now_h + 1) * h, now_w * w: (now_w + 1) * w, :] = img
    now_w += 1
    if now_w == cnt_w:
        now_w = 0
        now_h += 1

cv2.imwrite('res.png', res)

True

![](https://ai-studio-static-online.cdn.bcebos.com/719712e91eae44269498c55ef3786a93de1882155c284b72b6a9f57c056262c1)
